In [3]:
%pip install keras_preprocessing

  Obtaining dependency information for keras_preprocessing from https://files.pythonhosted.org/packages/79/4c/7c3275a01e12ef9368a892926ab932b33bb13d55794881e3573482b378a7/Keras_Preprocessing-1.1.2-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 253.8 kB/s eta 0:00:00--:--
Note: you may need to restart the kernel to use updated packages.


In [7]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [17]:
TRAIN_DIR = '/Users/constantinwiederin/Documents/IE/Computer vision/images 2/train'
TEST_DIR = '/Users/constantinwiederin/Documents/IE/Computer vision/images 2/validation'

In [19]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        if label == '.DS_Store':
            continue
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)
print(train)

happy completed
sad completed
fear completed
surprise completed
neutral completed
angry completed
disgust completed
                                                   image    label
0      /Users/constantinwiederin/Documents/IE/Compute...    happy
1      /Users/constantinwiederin/Documents/IE/Compute...    happy
2      /Users/constantinwiederin/Documents/IE/Compute...    happy
3      /Users/constantinwiederin/Documents/IE/Compute...    happy
4      /Users/constantinwiederin/Documents/IE/Compute...    happy
...                                                  ...      ...
28816  /Users/constantinwiederin/Documents/IE/Compute...  disgust
28817  /Users/constantinwiederin/Documents/IE/Compute...  disgust
28818  /Users/constantinwiederin/Documents/IE/Compute...  disgust
28819  /Users/constantinwiederin/Documents/IE/Compute...  disgust
28820  /Users/constantinwiederin/Documents/IE/Compute...  disgust

[28821 rows x 2 columns]


In [20]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)
print(test)

happy completed
sad completed
fear completed
surprise completed
neutral completed
angry completed
disgust completed
                                                  image    label
0     /Users/constantinwiederin/Documents/IE/Compute...    happy
1     /Users/constantinwiederin/Documents/IE/Compute...    happy
2     /Users/constantinwiederin/Documents/IE/Compute...    happy
3     /Users/constantinwiederin/Documents/IE/Compute...    happy
4     /Users/constantinwiederin/Documents/IE/Compute...    happy
...                                                 ...      ...
7061  /Users/constantinwiederin/Documents/IE/Compute...  disgust
7062  /Users/constantinwiederin/Documents/IE/Compute...  disgust
7063  /Users/constantinwiederin/Documents/IE/Compute...  disgust
7064  /Users/constantinwiederin/Documents/IE/Compute...  disgust
7065  /Users/constantinwiederin/Documents/IE/Compute...  disgust

[7066 rows x 2 columns]


In [21]:
from tqdm.notebook import tqdm

In [22]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale=True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

train_features = extract_features(train['image'])
test_features = extract_features(test['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

/Users/constantinwiederin/anaconda3/lib/python3.11/site-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


  0%|          | 0/7066 [00:00<?, ?it/s]

In [23]:
x_train = train_features / 255.0
x_test = test_features / 255.0

In [24]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [27]:
# sequential model
model = Sequential()

# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# flattening
model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(7, activation='softmax'))

# model compilation
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 197s 867ms/step - accuracy: 0.2343 - loss: 1.8383 - val_accuracy: 0.2583 - val_loss: 1.8038
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 190s 842ms/step - accuracy: 0.2501 - loss: 1.8034 - val_accuracy: 0.2819 - val_loss: 1.7335
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 218s 967ms/step - accuracy: 0.2830 - loss: 1.7341 - val_accuracy: 0.3596 - val_loss: 1.6156
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 232s 1s/step - accuracy: 0.3545 - loss: 1.6272 - val_accuracy: 0.4254 - val_loss: 1.4763
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 240s 1s/step - accuracy: 0.3980 - loss: 1.5252 - val_accuracy: 0.4611 - val_loss: 1.3841
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 230s 1s/step - accuracy: 0.4272 - loss: 1.4719 - val_accuracy: 0.4810 - val_loss: 1.3629
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 225s 994ms/step - accuracy: 0.4488 - loss: 1.4219 - val_accuracy: 0.4948 - val_loss: 1.3022
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 226s 1s/step - accuracy: 0.4684 - los

In [29]:
model_json = model.to_json()
with open("emotiondetector.json", 'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [4]:
from keras.models import model_from_json
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

/Users/constantinwiederin/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [5]:
label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
def ef(image):
    img = load_img(image, grayscale=True)
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

In [8]:
image = '/Users/constantinwiederin/Documents/IE/Computer vision/images 2/images/train/surprise/55.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ", pred_label)

original image is of sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
model prediction is  surprise


/Users/constantinwiederin/anaconda3/lib/python3.11/site-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ", pred_label)
plt.imshow(img.reshape(48, 48), cmap='gray')

In [35]:
import cv2
from keras.models import model_from_json
import numpy as np

# Load the pre-trained model architecture from JSON file
json_file = open("facialemotionmodel.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)

# Load the pre-trained model weights
model.load_weights("facialemotionmodel.h5")

# Load the Haar cascade classifier for face detection
haar_file = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)

# Define a function to extract features from an image
def extract_features(image):
    feature = np.array(image)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

# Open the webcam (camera)
webcam = cv2.VideoCapture(0)

# Define labels for emotion classes
labels = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

while True:
    # Read a frame from the webcam
    i, im = webcam.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(im, 1.3, 5)

    try:
        # For each detected face, perform facial emotion recognition
        for (p, q, r, s) in faces:
            # Extract the region of interest (ROI) which contains the face
            image = gray[q:q + s, p:p + r]

            # Draw a rectangle around the detected face
            cv2.rectangle(im, (p, q), (p + r, q + s), (255, 0, 0), 2)

            # Resize the face image to the required input size (48x48)
            image = cv2.resize(image, (48, 48))

            # Extract features from the resized face image
            img = extract_features(image)

            # Make a prediction using the trained model
            pred = model.predict(img)

            # Get the predicted label for emotion
            prediction_label = labels[pred.argmax()]

            # Display the predicted emotion label near the detected face
            cv2.putText(im, f'Emotion: {prediction_label}', (p - 10, q - 10),
                        cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255))

        # Display the frame with annotations in real-time
        cv2.imshow("Real-time Facial Emotion Recognition", im)

        # Break the loop if the 'Esc' key is pressed
        if cv2.waitKey(1) == 27:
            break

    except cv2.error:
        pass

# Release the webcam and close all OpenCV windows
webcam.release()
cv2.destroyAllWindows()

FileNotFoundError: [Errno 2] No such file or directory: 'facialemotionmodel.json'